In [4]:
import pandas as pd
from tqdm import tqdm
from tqdm.notebook import tqdm
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 300)

In [5]:
#Set Chunk Number
chunk = 5

In [ ]:
#Load in Docket Data 
columns_to_load = ['id', 'court_id']  # Adjust this list based on your actual column names
try:
    # Reading the CSV file into a DataFrame
    filtered_docket_df = pd.read_csv('/vast/amr10211/dockets-2023-08-31-filtered-withna.csv', 
                                     usecols=columns_to_load, 
                                     error_bad_lines=False,
                                     warn_bad_lines=True)
except pd.errors.ParserError as e:
    print(f"ParserError: {e}")

#Load in Cluster Data
columns_to_load = ['id', 'docket_id']
cluster_df =  pd.read_csv('/vast/amr10211/opinions-cluster-data-lc.csv', usecols=columns_to_load)



In [23]:
#Load Inference Data
inference_df = pd.read_csv('/scratch/amh9750/capstone/bert_inference/inference_results_legal_BERT_chunk_5.csv')
inference_df = inference_df.iloc[:20000] #REMOVE THIS

/state/partition1/job-40646907/ipykernel_621217/1878129563.py:2: DtypeWarning: Columns (15,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  inference_df = pd.read_csv('/scratch/amh9750/capstone/bert_inference/inference_results_legal_BERT_chunk_5.csv')


In [24]:
#Merge Inference with Cluster to get Docket_ID and then Docket to get Court_ID
inference_df = inference_df.rename(columns={'id': 'opinion_id'})
cluster_df = cluster_df.rename(columns={'id': 'cluster_id'})
inference_df = inference_df.merge(cluster_df[['cluster_id','docket_id']], how='left', on='cluster_id')
inference_df = inference_df.merge(filtered_docket_df, how='left', left_on='docket_id', right_on='id')


In [25]:
#Read Remaining Data
people_df = pd.read_csv('/vast/amr10211/people-db-people-2023-08-31.csv.bz2')
people_positions_df = pd.read_csv('/vast/amr10211/people-db-positions-2023-08-31.csv.bz2')
president_df = pd.read_csv('/vast/amr10211/president_metadata.csv')

#Reformat IDs
people_df = people_df.rename(columns={'id': 'person_id'})
people_positions_df = people_positions_df.merge(people_df, how='left', on='person_id')

# Merge DataFrames based on 'author_id' and 'person_id'
merged_df = pd.merge(inference_df, people_df, left_on='author_id', right_on='person_id', how='left')

# Fill missing values in 'imputed_author_str' with 'name_last'
merged_df['imputed_author_str'] = merged_df['author_str'].fillna(merged_df['name_last'])

# Drop redundant 'person_id' column if needed
merged_df = merged_df.drop('person_id', axis=1)

# Update the original inference_df with the changes
inference_df['imputed_author_str'] = merged_df['imputed_author_str']

# Convert 'date_start' and 'date_termination' columns to datetime objects
people_positions_df['date_start'] = pd.to_datetime(people_positions_df['date_start'], format='%Y-%m-%d', errors='coerce')
people_positions_df['date_termination'] = pd.to_datetime(people_positions_df['date_termination'], format='%Y-%m-%d', errors='coerce')
inference_df['date_filed'] = pd.to_datetime(inference_df['date_filed'], format='%Y-%m-%d', errors='coerce')

# Filter people for only justices and presidents
people_positions_df = people_positions_df[people_positions_df['position_type'].str.contains('jus|jud|mag|pres', case=False, regex=True, na=False)]

# Filter for termination date greater than 1930 and null (not terminated)
people_positions_df = people_positions_df[(people_positions_df['date_termination'].dt.year >= 1930) | pd.isnull(people_positions_df['date_termination'])]
president_df['date_start'] = pd.to_datetime(president_df['date_start'])
president_df['date_termination'] = pd.to_datetime(president_df['date_termination'])

# Change author name column name
inference_df['imputed_author_str'] = inference_df['imputed_author_str'].fillna(inference_df['imputed_column'])
inference_df = inference_df.drop('imputed_column', axis=1)

#Make blank person_id columns
inference_df['imputed_person_id'] = inference_df['author_id']


/state/partition1/job-40646907/ipykernel_621217/3759352766.py:3: DtypeWarning: Columns (11,12,13,14,22,23,25) have mixed types. Specify dtype option on import or set low_memory=False.
  people_positions_df = pd.read_csv('/vast/amr10211/people-db-positions-2023-08-31.csv.bz2')


In [26]:
people_positions_df[people_positions_df['person_id']==7168]

,id,date_created_x,date_modified_x,position_type,job_title,sector,organization_name,location_city,location_state,date_nominated,date_elected,date_recess_appointment,date_referred_to_judicial_committee,date_judicial_committee_action,judicial_committee_action,date_hearing,date_confirmation,date_start,date_granularity_start,date_termination,termination_reason,date_granularity_termination,date_retirement,nomination_process,vote_type,voice_vote,votes_yes,votes_no,votes_yes_percent,votes_no_percent,how_selected,has_inferred_values,appointer_id,court_id,person_id,predecessor_id,school_id,supervisor_id,date_created_y,date_modified_y,date_completed,fjc_id,slug,name_first,name_middle,name_last,name_suffix,date_dob,date_granularity_dob,date_dod,date_granularity_dod,dob_city,dob_state,dob_country,dod_city,dod_state,dod_country,gender,religion,ftm_total_received,ftm_eid,has_photo,is_alias_of_id
30688,31387,2016-04-20 15:28:05.485187+00,2016-04-20 15:28:05.485207+00,jud,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1976-05-03,%Y-%m-%d,2000-01-01,NaN,%Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,gactapp,7168,NaN,NaN,NaN,2016-04-20 15:28:05.476267+00,2020-11-25 16:30:09.267808+00,NaN,NaN,william-leroy-mcmurray-jr,William,Leroy,McMurray,jr,NaN,NaN,NaN,NaN,NaN,NaN,United States,NaN,NaN,United States,NaN,NaN,NaN,NaN,f,NaN


In [27]:
def find_person_id(row, people_df, people_positions_df):
    '''Logic to impute person_id given multiple criteria in row'''
    # 1. If row['author_id'] exists then return row['author_id']
    # print(row['imputed_author_str'])
    if pd.notnull(row['author_id']):
        return row['author_id']

    # 2. If there is only one people_df['name_last'] that matches row['author_str1']
    name_last_match = people_df[people_df['name_last'] == row['imputed_author_str']]
    name_last_match_count = name_last_match.groupby('name_last').size()
    if name_last_match_count.get(row['imputed_author_str'], 0) == 1:
        return int(name_last_match['person_id'].values[0])

    # 3. If there are multiple people_df['name_last'] that matches row['author_str1']
    multiple_name_last_match = name_last_match_count[name_last_match_count > 1].index
    for name_last in multiple_name_last_match:
        candidate_set = people_positions_df[people_positions_df['name_last'] == name_last]

        # 3a. Check if row['date_filed'] year is between people_positions_df['date_start'] and people_positions_df['date_termination']
        date_match = candidate_set[
            (
                (candidate_set['date_start'].dt.year.le(row['date_filed'].year) | candidate_set['date_start'].isnull()) &
                (
                    candidate_set['date_termination'].dt.year.ge(row['date_filed'].year) | candidate_set['date_termination'].isnull()
                ) &
                (candidate_set['name_last'] == row['imputed_author_str'])
            )
        ]
        if len(date_match) == 1:
            return int(date_match['person_id'].values[0])
        elif len(date_match) > 1:
            candidate_set = date_match

        # 3b. Check if row['imputed_court_id'] = people_df['imputed_court_id']
        court_id_match = candidate_set[(candidate_set['court_id'] == row['imputed_court_id']) & (candidate_set['name_last'] == row['imputed_author_str'])]
        if len(court_id_match) == 1:
            # print('YAY! found from court id')
            return court_id_match['person_id'].values[0]
        elif len(court_id_match) > 1:
            candidate_set = court_id_match
            
        # 3c. Check if there is a match based on date range and last name
        court_date_name_match = candidate_set[
            ((candidate_set['date_start'].le(row['date_filed']) | candidate_set['date_start'].isnull()) &
             (candidate_set['date_termination'].ge(row['date_filed']) | candidate_set['date_termination'].ge(row['date_filed']) | candidate_set['date_termination'].isnull()) &
             (candidate_set['name_last'] == row['imputed_author_str']))
        ]

        if len(court_date_name_match) == 1:
            return court_date_name_match['person_id'].values[0]
    

    # 4. If there are zero people_df['name_last'] that matches row['author_str1'], state error
    return None  # You may want to return a default value or handle the error as per your needs



def fill_missing_person_id(row, df, proximity_range=1000):
    '''Logic to impute person_id given neighbors with same author_str'''
    if pd.isnull(row['imputed_person_id']):
        start_index = max(0, row.name - proximity_range)
        end_index = min(len(df), row.name + proximity_range + 1)

        nearby_data = df.loc[start_index:end_index]
        matching_row = nearby_data.loc[~pd.isnull(nearby_data['imputed_person_id']) & (nearby_data['imputed_author_str'] == row['imputed_author_str'])]

        if not matching_row.empty:
            # Use the imputed_person_id of the first matching row found in the proximity
            return matching_row.iloc[0]['imputed_person_id']

    return row['imputed_person_id']

def fill_missing_court_id(row, df, proximity_range=1000):
    '''Logic to impute court_id given neighbors with same author_str'''
    if pd.isnull(row['imputed_court_id']):
        start_index = max(0, row.name - proximity_range)
        end_index = min(len(df), row.name + proximity_range + 1)

        nearby_data = df.loc[start_index:end_index]
        matching_row = nearby_data.loc[~pd.isnull(nearby_data['imputed_court_id']) & (nearby_data['imputed_author_str'] == row['imputed_author_str'])]

        if not matching_row.empty:
            # Use the court_id of the first matching row found in the proximity
            return matching_row.iloc[0]['imputed_court_id']

    return row['imputed_court_id']


In [28]:
# Apply the functions above to each row in the DataFrame
inference_df['imputed_court_id'] = inference_df['court_id']
inference_df['imputed_court_id'] = inference_df.apply(lambda row: fill_missing_court_id(row, inference_df), axis=1)
inference_df['imputed_person_id'] = inference_df.apply(find_person_id, axis=1, people_df=people_df, people_positions_df = people_positions_df)
inference_df['imputed_person_id'] = pd.to_numeric(inference_df['imputed_person_id'], errors='coerce').astype('Int64')
inference_df['imputed_person_id'] = inference_df.apply(lambda row: fill_missing_person_id(row, inference_df), axis=1)

# Create the new Unique IDs based on conditions
inference_df['unique_person_id'] = inference_df.apply(lambda row:
    str(row['imputed_person_id']) if not pd.isnull(row['imputed_person_id'])
    else f"{row['imputed_author_str']}_{row['imputed_court_id']}" if not pd.isnull(row['imputed_author_str']) and not pd.isnull(row['imputed_court_id'])
    else str(row['imputed_author_str']), axis=1
)


In [29]:
inference_df.count()

Unnamed: 0            20000
resource_uri              0
opinion_id            20000
absolute_url              0
cluster_id            20000
cluster                   0
author_id              7288
author                    0
joined_by                 0
date_created          20000
date_modified         20000
author_str            14117
per_curiam            20000
joined_by_str             0
type                  20000
sha1                  14372
page_count             4532
download_url           4800
local_path             6916
extracted_by_ocr      20000
opinions_cited            0
judges                19982
judge                 20000
date_filed            20000
token_count           20000
predicted_label       20000
probability           20000
docket_id             20000
id                    11293
court_id              11293
imputed_author_str    20000
imputed_person_id     15941
imputed_court_id      15192
unique_person_id      20000
dtype: int64

In [30]:

inference_df['year_filed'] = inference_df['date_filed'].dt.year

position_date_df = pd.merge(
    inference_df,
    people_positions_df[['person_id', 'appointer_id', 'date_start', 'date_termination']],
    how='left',
    left_on='imputed_person_id',
    right_on='person_id'
)
position_date_df = position_date_df.reset_index(drop=True)

In [34]:
people_positions_df[people_positions_df['person_id']==5411]

,id,date_created_x,date_modified_x,position_type,job_title,sector,organization_name,location_city,location_state,date_nominated,date_elected,date_recess_appointment,date_referred_to_judicial_committee,date_judicial_committee_action,judicial_committee_action,date_hearing,date_confirmation,date_start,date_granularity_start,date_termination,termination_reason,date_granularity_termination,date_retirement,nomination_process,vote_type,voice_vote,votes_yes,votes_no,votes_yes_percent,votes_no_percent,how_selected,has_inferred_values,appointer_id,court_id,person_id,predecessor_id,school_id,supervisor_id,date_created_y,date_modified_y,date_completed,fjc_id,slug,name_first,name_middle,name_last,name_suffix,date_dob,date_granularity_dob,date_dod,date_granularity_dod,dob_city,dob_state,dob_country,dod_city,dod_state,dod_country,gender,religion,ftm_total_received,ftm_eid,has_photo,is_alias_of_id
26856,27459,2016-04-20 15:26:17.579279+00,2016-04-20 15:26:17.5793+00,jud,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1977-01-01,%Y,1984-01-31,resign,%Y-%m-%d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,oklacrimapp,5411,NaN,NaN,NaN,2016-04-20 15:26:17.568935+00,2020-11-25 16:30:09.267808+00,NaN,NaN,tom-r-cornish,Tom,R.,Cornish,NaN,1944-08-22,%Y-%m-%d,NaN,NaN,NaN,NaN,United States,NaN,NaN,United States,m,NaN,NaN,NaN,f,NaN
35538,35730,2020-08-05 22:23:05.500038+00,2020-08-05 22:23:05.500061+00,jud,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,okeb,5411,NaN,NaN,NaN,2016-04-20 15:26:17.568935+00,2020-11-25 16:30:09.267808+00,NaN,NaN,tom-r-cornish,Tom,R.,Cornish,NaN,1944-08-22,%Y-%m-%d,NaN,NaN,NaN,NaN,United States,NaN,NaN,United States,m,NaN,NaN,NaN,f,NaN


In [31]:
position_date_df[position_date_df['imputed_person_id'].notnull() & position_date_df['date_start'].isnull()]

,Unnamed: 0,resource_uri,opinion_id,absolute_url,cluster_id,cluster,author_id,author,joined_by,date_created,date_modified,author_str,per_curiam,joined_by_str,type,sha1,page_count,download_url,local_path,extracted_by_ocr,opinions_cited,judges,judge,date_filed,token_count,predicted_label,probability,docket_id,id,court_id,imputed_author_str,imputed_person_id,imputed_court_id,unique_person_id,year_filed,person_id,appointer_id,date_start,date_termination
269,172,NaN,1666132,NaN,1666132,NaN,NaN,NaN,NaN,2013-10-30 07:04:05.016723+00,2023-08-24 01:17:20.546386+00,Woodruff,f,NaN,010combined,4aff0a6524c678a494988c840c70e2cbb8237cd7,NaN,NaN,NaN,f,NaN,Woodruff,Woodruff,1958-04-17,5880,1,0.872685,1254675,1254675.0,texapp,Woodruff,3546,texapp,3546,1958,NaN,NaN,NaT,NaT
762,465,NaN,9540340,NaN,2057085,NaN,NaN,NaN,NaN,2023-08-07 16:14:50.282325+00,2023-08-07 16:14:50.282337+00,English,f,NaN,040dissent,NaN,NaN,NaN,NaN,f,NaN,"Drucker, English",English,1966-06-17,751,1,0.996229,1899942,NaN,NaN,English,999,NaN,999,1966,NaN,NaN,NaT,NaT
1591,959,NaN,9583699,NaN,1251248,NaN,NaN,NaN,NaN,2023-08-21 22:41:21.372431+00,2023-08-21 22:41:21.372453+00,Gardebring,f,NaN,020lead,NaN,NaN,NaN,NaN,f,NaN,"Blatz, Gardebring, Keith",Gardebring,1998-01-28,3279,1,0.943753,1245241,1245241.0,minn,Gardebring,4837,minn,4837,1998,NaN,NaN,NaT,NaT
9163,5931,NaN,1157123,NaN,1157123,NaN,5411.0,NaN,NaN,2013-10-30 04:26:47.160383+00,2023-08-07 16:12:41.453417+00,Cornish,f,NaN,010combined,1bda461d543084dd3f04a24a2cd64a8de9cdbb11,NaN,NaN,NaN,f,NaN,"Brett, Bussey, Cornish",Cornish,1981-07-30,3262,1,0.886477,555385,NaN,NaN,Cornish,5411,NaN,5411,1981,5411.0,NaN,NaT,NaT
9702,6316,NaN,9666778,NaN,1668624,NaN,NaN,NaN,NaN,2023-08-24 01:27:21.986616+00,2023-08-24 01:27:21.986625+00,Gardebring,f,NaN,035concurrenceinpart,NaN,NaN,NaN,NaN,f,NaN,"Gardebring, Simonett, Wahl",Gardebring,1994-06-30,113,1,0.889252,1256908,1256908.0,minn,Gardebring,4837,minn,4837,1994,NaN,NaN,NaT,NaT
10426,6838,NaN,2056847,NaN,2056847,NaN,NaN,NaN,NaN,2013-10-30 08:12:02.278367+00,2023-08-07 16:13:54.418367+00,English,f,NaN,010combined,a1821ecb519151f8a943cb30697a3704ae54abae,NaN,NaN,NaN,f,NaN,English,English,1966-06-22,2979,0,0.945915,1899372,NaN,NaN,English,999,NaN,999,1966,NaN,NaN,NaT,NaT
10832,7106,NaN,2056894,NaN,2056894,NaN,NaN,NaN,NaN,2013-10-30 08:12:02.700397+00,2023-08-07 16:14:04.788367+00,English,f,NaN,010combined,d91989eef85f23a0d8a9246dbd61764496861f9f,NaN,NaN,NaN,f,NaN,English,English,1966-04-07,312,0,0.987620,2541982,NaN,NaN,English,999,NaN,999,1966,NaN,NaN,NaT,NaT
10892,7142,NaN,2056903,NaN,2056903,NaN,NaN,NaN,NaN,2013-10-30 08:12:02.787763+00,2023-08-07 16:14:07.014482+00,English,f,NaN,010combined,469fe9a5164d491c9ebda2e9be3bdad5c0266335,NaN,NaN,NaN,f,NaN,English,English,1966-04-22,7535,0,0.944284,1899576,NaN,NaN,English,999,NaN,999,1966,NaN,NaN,NaT,NaT
11064,7272,NaN,2056934,NaN,2056934,NaN,NaN,NaN,NaN,2013-10-30 08:12:03.023658+00,2023-08-07 16:14:13.970686+00,English,f,NaN,010combined,6e53b49a8a91e98f741b3d47479aa6c443e3f9d7,NaN,NaN,NaN,f,NaN,English,English,1966-06-10,270,0,0.935374,2572734,NaN,NaN,English,999,NaN,999,1966,NaN,NaN,NaT,NaT
12008,7949,NaN,1157872,NaN,1157872,NaN,NaN,NaN,NaN,2013-10-30 04:27:00.939052+00,2023-08-07 16:16:51.830315+00,Patterson,f,NaN,010combined,fa36afecbe303ea8fc52648e8ec49323958560f1,NaN,NaN,NaN,f,NaN,"De Concini, Honorable, Patterson, Phelps, Prad...",Patterson,1950-11-06,3080,0,0.785957,556188,NaN,NaN,Patterson,15946,NaN,15946,1950,15946.0,NaN,NaT,NaT


In [86]:


# Filtering by date if date granularity for start and termination, or year if year granularity (jan 1 date)

position_date_df = position_date_df[
    (
        (
            (
                (
                    (position_date_df['date_start'].dt.month != 1) & 
                    (position_date_df['date_start'].dt.day != 1) & 
                    (
                    (position_date_df['date_termination'].dt.month != 1) & 
                    (position_date_df['date_termination'].dt.day != 1) |
                    (position_date_df['date_termination'].isnull())     
                    )    
                ) & (
                    (position_date_df['date_filed'] >= position_date_df['date_start']) & 
                    (position_date_df['date_filed'] <= position_date_df['date_termination'])
                )
            ) | (
                (
                    (position_date_df['date_start'].dt.month == 1) & 
                    (position_date_df['date_start'].dt.day == 1) & 
                    (
                    (position_date_df['date_termination'].dt.month == 1) & 
                    (position_date_df['date_termination'].dt.day == 1) |
                    (position_date_df['date_termination'].isnull())
                    )    
                    
                ) & 
                    (position_date_df['year_filed'] >= position_date_df['date_start'].dt.year) & 
                    (position_date_df['year_filed'] <= position_date_df['date_termination'].dt.year)
                 )
             )
         )
    | (position_date_df['imputed_person_id'].isnull()) 
    | (position_date_df['date_start'].isnull()) 
    | (position_date_df['date_termination'].isnull())
]

# Dropping the temporary 'year_filed' column
position_date_df = position_date_df.drop(columns=['year_filed'])
position_date_df = position_date_df[['opinion_id','appointer_id','date_start','date_termination']].drop_duplicates().reset_index(drop=True)

#Merge back with original inference_df to get back missing rows
visualization_df = inference_df.merge(position_date_df, how='left', on=['opinion_id'])
# Sort by 'date_start' in ascending order
visualization_df = visualization_df.drop_duplicates(subset=['opinion_id'], keep='last').reset_index(drop=True) #For any remaining duplicates, keep final record in people_positions_df



In [87]:
visualization_df.count()

Unnamed: 0              64
resource_uri             0
opinion_id              64
absolute_url             0
cluster_id              64
cluster                  0
author_id               31
author                   0
joined_by                0
date_created            64
date_modified           64
author_str               0
per_curiam              64
joined_by_str            0
type                    64
sha1                    64
page_count              64
download_url            61
local_path              64
extracted_by_ocr        64
opinions_cited           0
judges                  64
judge                   64
date_filed              64
token_count             64
predicted_label         64
probability_class_0     64
probability_class_1     64
probability_class_2     64
probability_class_3     64
probability_class_4     64
probability_class_5     64
probability_class_6     64
probability_class_7     64
probability_class_8     64
probability_class_9     64
probability_class_10    64
p

In [88]:
def impute_president(row):
    '''Imputes appointing president based on judge start date'''
    # Check if president_id is not null, if yes, return original values
    if pd.notnull(row['president_id']):
        return row['president_id'], row['president_name']
    
    # If president_id is null, look up in president_df based on date conditions
    president_match = president_df[
        (president_df['date_start'] <= row['date_start']) &
        (row['date_start'] <= president_df['date_termination'])
    ]
    
    # If there's a match, return the imputed values
    if not president_match.empty:
        return president_match.iloc[0]['president_id'], president_match.iloc[0]['president_name']
    
    # If no match is found, return null values or any other default values as needed
    return None, None



In [89]:
# Merge with president_df to get populated presidents
visualization_df = visualization_df.merge(president_df[['president_id', 'president_name', 'partisanship']], how='left', left_on='appointer_id', right_on='president_id')


# Apply the imputation function to create the new columns
visualization_df[['imputed_president_id', 'imputed_president_name']] = visualization_df.apply(impute_president, axis=1, result_type='expand')

# Perform the second merge
visualization_df = visualization_df.merge(president_df[['president_id', 'partisanship']], how='left', left_on='imputed_president_id', right_on='president_id')

# Fill NaN values in 'partisanship' from the second merge with values from the first merge
visualization_df['partisanship'] = visualization_df['partisanship_y'].fill_na(visualization_df['partisanship_x'])

# Drop unnecessary columns
visualization_df = visualization_df.drop(['partisanship_x', 'partisanship_y'], axis=1)
# visualization_df.to_csv(f'/vast/amr10211/visualization_results_legal_BERT_chunk_{chunk}.csv')


In [90]:
visualization_df.count()

Unnamed: 0                64
resource_uri               0
opinion_id                64
absolute_url               0
cluster_id                64
cluster                    0
author_id                 31
author                     0
joined_by                  0
date_created              64
date_modified             64
author_str                 0
per_curiam                64
joined_by_str              0
type                      64
sha1                      64
page_count                64
download_url              61
local_path                64
extracted_by_ocr          64
opinions_cited             0
judges                    64
judge                     64
date_filed                64
token_count               64
predicted_label           64
probability_class_0       64
probability_class_1       64
probability_class_2       64
probability_class_3       64
probability_class_4       64
probability_class_5       64
probability_class_6       64
probability_class_7       64
probability_cl

In [83]:
visualization_df[visualization_df['imputed_president_name'].notnull() & visualization_df['partisanship'].isnull()]


KeyError: 'partisanship'